# Environment

In [2]:
## Imports and data loading

%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras import metrics

import gym

from musicrl.midi2vec import MidiVectorMapper, PostProcessor
from musicrl.render import *
from musicrl.data import RandomMidiDataGenerator
from musicrl import mel_lstm

import pretty_midi
from glob import glob


REAL = 1
GEN = 0

na = None # new axis

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
filepaths = list(glob('maestro-v2.0.0/2008/**.midi'))
real_midis = [pretty_midi.PrettyMIDI(i) for i in filepaths]
mapper = MidiVectorMapper(real_midis)